<img src="https://drive.google.com/uc?export=view&id=1aeor4TdnZx3rovf30GoegUWP9Qcj6l6w" width="100%">

# **Proyecto Aplicado: Definición de la Aplicación**
---



In [1]:
!pip install -q openai langchain
!pip install -q langchain_experimental
!pip install -q kaleido python-multipart cohere tiktoken openai langchain chromadb lark
!pip install -q transformers[torch] xformers sentencepiece sacremoses datasets tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 993.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset, Dataset
from IPython.display import display
from transformers import (
        CodeGenTokenizer, CodeGenConfig, CodeGenForCausalLM,
        TrainingArguments, Trainer, DataCollatorForLanguageModeling,
        pipeline
        )

## **1. Metodología Propuesta**
---

En esta sección deberá presentar un diagrama con la arquitectura propuesta para la aplicación basada en LLMs:

[diagrama]

- *Conjunto de datos*: [IMDB movies](https://www.kaggle.com/datasets/ashpalsingh1525/imdb-movies-dataset/data) El conjunto de datos IMDB contiene información sobre películas, incluidos sus nombres, fechas de estreno, valoraciones de usuarios, géneros, descripciones generales, miembros del reparto y del equipo, títulos originales, estado de producción, idiomas originales, presupuestos, ingresos y países de origen. El tamaño del dataframe es de aproximadamente 10000 registros de películas únicas.
- *Manejo de datos*: Teniendo en cuenta que el conjunto de datos a utilizar proviene de Kaggle y que todos los campos resultan relevantes para el desarrollo del modelo, no se hace necesario el preprocesamiento exhaustivo de los datos. Lo único que realizaremos es un una eliminación de los campos nulos. Adicionalmente, el conjunto de datos será convertido a un dataframe de pandas para poder usarlo directamente.
- *Modelamiento*: En fase exploratoria vamos a estar usando un model de OpenAI (gpt-3.5-turbo-instruct). Ya en fases más avanzadas del proyecto podríamos usar un modelo de Huggingface para tener mayor libertad.

## **2. Carga de datos**
---


In [5]:
# INGRESE SU CODIGO AQUI
from google.colab import drive
drive.mount('/content/drive')
path="/content/drive/MyDrive/Proyecto curso LLMs/0. Data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
df = pd.read_csv(path+'imdb_movies.csv')

In [7]:
df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


Se removerán todas las filas con valores nulos

In [8]:
df = df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10052 entries, 0 to 10051
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   names       10052 non-null  object 
 1   date_x      10052 non-null  object 
 2   score       10052 non-null  float64
 3   genre       10052 non-null  object 
 4   overview    10052 non-null  object 
 5   crew        10052 non-null  object 
 6   orig_title  10052 non-null  object 
 7   status      10052 non-null  object 
 8   orig_lang   10052 non-null  object 
 9   budget_x    10052 non-null  float64
 10  revenue     10052 non-null  float64
 11  country     10052 non-null  object 
dtypes: float64(3), object(9)
memory usage: 942.5+ KB


## **3. Implementación modelos**
---


### **3.1. RECOMENDADORES**
---

#### **3.1.1. Con agente de Pandas**

Vamos a importar el modelo de OpenAI:

In [9]:
from langchain.llms import OpenAI
from langchain.agents.agent_types import AgentType
from IPython.display import display

Definimos el modelo:

In [10]:
api_key = ""
llm = OpenAI(temperature=0.0, openai_api_key=api_key)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [11]:
llm.model_name

'gpt-3.5-turbo-instruct'

- **Inferencia**: agregue código que permita obtener el texto generado con el modelo sin entrenar.

In [12]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

Vamos a crear el agente a partir de un LLM, del `DataFrame` y del tipo de agente:

In [13]:
agent = create_pandas_dataframe_agent(
        llm=llm,
        df=df,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
        )

In [14]:
print(agent.run("What is the movie of romance genre that do you recommend more? Please only recommend one movie"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should filter the dataframe by genre and then sort by score to find the highest rated romance movie.
Action: python_repl_ast
Action Input: df[df['genre'].str.contains('Romance')].sort_values(by='score', ascending=False).head(1)            names       date_x  score    genre  \
8128  Housekeeper  03/26/2020   100.0  Romance   

                                               overview  \
8128  Min-woo, the consulate general of the overseas...   

                                                   crew orig_title     status  \
8128  Ahn So-hee, , Min Do-yoon, , Kang Min-woo, , H...       예쁜식모   Released   

     orig_lang     budget_x       revenue country  
8128    Korean  201000000.0  1.569324e+09      KR  100.0 is a perfect score, so this is the highest rated romance movie in the dataframe.
Final Answer: The movie of romance genre that I recommend the most is "Housekeeper".

> Finished chain.
The movie of romance genre that I recommend t

#### **3.1.2. Con embedding y Vector Store**

Creación del documento:

In [15]:
from langchain.schema import Document

def get_document(data):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    overview = data.pop("overview")
    doc = Document(
            page_content=overview,
            metadata=data
            )
    return doc
    ### FIN DEL CÓDIGO ###

In [16]:
data = df.to_dict(orient="records")
data = list(map(get_document, data))

Modelo de Embedding:

In [17]:
from langchain.embeddings import OpenAIEmbeddings

def get_embedding(api_key):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    embedding = OpenAIEmbeddings(openai_api_key=api_key)
    return embedding
    ### FIN DEL CÓDIGO ###

In [18]:
embedding = get_embedding(api_key)
display(embedding.model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


'text-embedding-ada-002'

In [19]:
embedding = get_embedding(api_key)

Creación de Vector Store con chromadb:

In [20]:
from langchain.vectorstores import Chroma

def get_db(data, embedding):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    db = Chroma.from_documents(
        documents=data,
        embedding=embedding
        )
    return db
    ### FIN DEL CÓDIGO ###

In [21]:
db = get_db(data, embedding)

Descripción de los atributos y del contenido del documento:

In [22]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [23]:
metadata_info = [
        AttributeInfo(
            name="names",
            description="name of the movie",
            type="string"
            ),
        AttributeInfo(
            name="date_x",
            description="date of release of the movie",
            type="int"
            ),
        AttributeInfo(
            name="score",
            description="score of the movie on a scale of 1 to 10",
            type="float"
            ),
        AttributeInfo(
            name="genre",
            description="genre of the movie",
            type="string"
            ),

        AttributeInfo(
            name="crew",
            description="names of the actors of the movie",
            type="string"
            ),

        AttributeInfo(
            name="orig_title",
            description="original name of the movie",
            type="string"
            ),

        AttributeInfo(
            name="status",
            description="movie release status",
            type="string"
            ),

        AttributeInfo(
            name="orig_lang",
            description="original language",
            type="string"
            ),

        AttributeInfo(
            name="budget_x",
            description="budget of the movie in dollars",
            type="string"
            ),

        AttributeInfo(
            name="revenue",
            description="revenue of the movie",
            type="string"
            ),
        AttributeInfo(
            name="country",
            description="country of origin of the film",
            type="string"
            )
        ]

Definir Auto-Recuperador

In [24]:
from langchain.llms import OpenAI

def get_retriever(
        api_key, vectorstore, metadata_info
        ):
    ### ESCRIBA SU CÓDIGO AQUÍ ###

    llm = OpenAI(temperature=0, openai_api_key=api_key)

    retriever = SelfQueryRetriever.from_llm(
        llm=llm,
        vectorstore=vectorstore,
        document_contents="overview of the movie",
        metadata_field_info=metadata_info,
        verbose=True
        )

    return retriever
    ### FIN DEL CÓDIGO ###

In [25]:
retriever = get_retriever(
        api_key=api_key,
        vectorstore=db,
        metadata_info = metadata_info
        )

Realizando consultas:

In [26]:
response = retriever.get_relevant_documents(
        query="I want a movie with high rate and romance genre?"
        )
display(response)

[Document(page_content='A man and woman meet and try to have a romantic affair, despite their personal problems and the interference of their disapproving friends.', metadata={'budget_x': 8500000.0, 'country': 'AU', 'crew': 'Demi Moore, Debbie, Rob Lowe, Danny Martin, Jim Belushi, Bernie Litgo, Elizabeth Perkins, Joan, George DiCenzo, Mr. Favio, Robin Thomas, Steve Carlson, Donna Gibbons, Alex, Megan Mullally, Pat, Michael Alldredge, Mother Malone', 'date_x': '08/14/1986 ', 'genre': 'Comedy,\xa0Drama,\xa0Romance', 'names': 'About Last Night...', 'orig_lang': ' English', 'orig_title': 'About Last Night...', 'revenue': 38702310.0, 'score': 62.0, 'status': ' Released'}),
 Document(page_content='In this romantic comedy, several friends, each dealing with unhappy love lives, turn to each other for help - but not always with the best results.', metadata={'budget_x': 49000000.0, 'country': 'NL', 'crew': 'Yolanthe Cabau, Bo, Victoria Koblenko, Cindy, Bo Maerten, Lisa, Géza Weisz, Jim, Edwin Jo

### **3.2. Predecir género a partir de overview**
---

#### **3.2.1. Modelo con solo prompting**

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
        ChatPromptTemplate, SystemMessagePromptTemplate,
        HumanMessagePromptTemplate, MessagesPlaceholder
        )

In [28]:
df_classification=df.copy()

In [29]:
df_classification['overview'] = df_classification['overview'].apply(lambda x: 'Overview: ' + x)
df_classification['classification(genre)'] = df_classification['genre'].str.split(',').str[0]

In [30]:
df_classification['classification(genre)'].unique()

array(['Drama', 'Science Fiction', 'Animation', 'Action', 'Thriller',
       'Horror', 'Comedy', 'War', 'Documentary', 'Crime', 'Family',
       'Adventure', 'Fantasy', 'Music', 'Mystery', 'Western', 'Romance',
       'TV Movie', 'History'], dtype=object)

In [31]:
df_classification[['overview','classification(genre)']].head(6)

,overview,classification(genre)
0,"Overview: After dominating the boxing world, A...",Drama
1,Overview: Set more than a decade after the eve...,Science Fiction
2,Overview: While working underground to fix a w...,Animation
3,Overview: Through a series of unfortunate even...,Animation
4,Overview: Good-hearted teenager William always...,Action
5,"Overview: Inspired by a true story, an oddball...",Thriller


In [32]:


def recommendation(overview):
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
  memory.chat_memory.add_user_message(
      '''
      Hello chat! I need you to predict the genre of a movie from the overview. I only want the information of the genre of the movie.
      The list of possible genres are these:
      ['Genre: Drama', 'Genre: Science Fiction', 'Genre: Animation', 'Genre: Action', 'Genre: Thriller',
        'Genre: Horror', 'Genre: Comedy', 'Genre: War', 'Genre: Documentary', 'Genre: Crime', 'Genre: Family',
        'Genre: Adventure', 'Genre: Fantasy', 'Genre: Music', 'Genre: Mystery', 'Genre: Western', 'Genre: Romance',
        'Genre: TV Movie', 'Genre: History']
      '''
      )
  memory.chat_memory.add_ai_message("Perfect! Give me the overview of the movie and i will give you the genre.")

  memory.chat_memory.add_user_message("Overview: Good-hearted teenager William always lived in hope of following in his late father’s footsteps and becoming a storm chaser. His father’s legacy has now been turned into a storm-chasing tourist business, managed by the greedy and reckless Zane Rogers, who is now using William as the main attraction to lead a group of unsuspecting adventurers deep into the eye of the most dangerous supercell ever seen.")
  memory.chat_memory.add_ai_message("Genre: Action")

  memory.chat_memory.add_user_message("Overview: Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure.")
  memory.chat_memory.add_ai_message("Genre: Science Fiction")


  template = HumanMessagePromptTemplate.from_template(
          template="{message}"
          )
  prompt = ChatPromptTemplate.from_messages(
          messages=[
              MessagesPlaceholder(variable_name="chat_history"),
              template
              ]
          )
  chain = LLMChain(
          prompt=prompt,
          llm=llm,
          memory=memory,
          )

  response = chain.run({"message": overview})
  return response

In [33]:
recommendation('''Overview: The united town of sheep and wolves lives a peaceful and quiet life until two unexpected guests turn up - a polar fox and a tiny ewe. No one ever expected them to bring a deadly danger, which can be overcome only if they work together. Only teamwork can solve big problems and deal with serious challenges - as there is strength in unity.''')

'\nAI: Genre: Animation'

In [34]:
df_classification_prueba_1=df_classification.iloc[10000:].reset_index(drop=True)
df_classification_prueba_1["Prediction"] = None
print("Número de filas del dataframe: ",len(df_classification_prueba_1))

Número de filas del dataframe:  52


In [35]:
import time
for i in range(len(df_classification_prueba_1)):
  if i%5==0:
    print("Step: ",i)
  overview_string=df_classification_prueba_1.iloc[i, 4]
  prediction = recommendation(overview_string)
  df_classification_prueba_1.loc[i, "Prediction"] = prediction
  time.sleep(1)

Step:  0
Step:  5
Step:  10
Step:  15
Step:  20
Step:  25
Step:  30
Step:  35
Step:  40
Step:  45
Step:  50


In [36]:
df_classification_prueba_1['Prediction_cleaned'] = df_classification_prueba_1['Prediction'].str.extract(r'Genre: (\w+)')
df_classification_prueba_1

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,classification(genre),Prediction,Prediction_cleaned
0,Tommy,03/26/1975,64.0,"Drama, Music, Fantasy","Overview: A psychosomatically deaf, dumb and b...","Oliver Reed, Frank, Ann-Margret, Nora, Roger D...",Tommy,Released,English,5000000.0,34251525.0,GB,Drama,\nAI: Genre: Drama,Drama
1,Acts of Violence,01/12/2018,57.0,"Action, Crime, Thriller",Overview: When his fiancee is kidnapped by hum...,"Bruce Willis, Detective James Avery, Cole Haus...",Acts of Violence,Released,English,10700000.0,386790.0,US,Action,\nAI: Genre: Thriller,Thriller
2,After Hours,09/13/1985,75.0,"Comedy, Thriller, Drama",Overview: An ordinary word processor has the w...,"Griffin Dunne, Paul Hackett, Rosanna Arquette,...",After Hours,Released,English,4500000.0,10609321.0,US,Comedy,\nAI: Genre: Thriller,Thriller
3,Small Fry,11/23/2011,69.0,"Animation, Family, Comedy",Overview: A fast food restaurant mini variant ...,"Tom Hanks, Woody (voice), Tim Allen, Buzz Ligh...",Small Fry,Released,English,116400000.0,187299322.0,US,Animation,\nAI: Genre: Animation,Animation
4,Four of the Apocalypse,08/12/1975,65.0,"Western, Action","Overview: Four petty criminals, three men and ...","Fabio Testi, Stubby Preston, Lynne Frederick, ...",I quattro dell'apocalisse,Released,Italian,73340000.0,552832145.2,IT,Western,\nAI: Genre: Western,Western
5,Love Wedding Repeat,04/10/2020,57.0,"Comedy, Romance",Overview: While trying to make his sister's we...,"Sam Claflin, Jack, Olivia Munn, Dina, Eleanor ...",Love Wedding Repeat,Released,English,128000000.0,419965614.6,GB,Comedy,\nAI: Genre: Comedy,Comedy
6,Brüno,07/08/2009,56.0,Comedy,Overview: Flamboyantly gay Austrian television...,"Sacha Baron Cohen, Brüno, Gustaf Hammarsten, L...",Brüno,Released,English,42000000.0,138708527.0,AU,Comedy,\nAI: Genre: Comedy,Comedy
7,Vixen!,10/15/1968,53.0,Drama,Overview: Vixen lives in a Canadian mountain r...,"Garth Pillsbury, Tom Palmer, Harrison Page, Ni...",Vixen!,Released,English,73000.0,8000000.0,US,Drama,"However, when she is raped by him, she can't ...",Drama
8,Lady Chatterley's Lover,09/06/2015,64.0,"Romance, Drama",Overview: An early-20th-century tale of love a...,"Richard Madden, Oliver Mellors, Holliday Grain...",Lady Chatterley's Lover,Released,English,102000000.0,254278668.2,GB,Romance,\nAI: Genre: Romance,Romance
9,Felony,08/28/2014,57.0,"Mystery, Thriller, Crime",Overview: Three detectives become embroiled in...,"Joel Edgerton, Detective Malcolm Toohey, Jai C...",Felony,Released,English,169600000.0,453824304.6,AU,Mystery,\nAI: Genre: Thriller,Thriller


In [39]:
def check_contains1(row):
    return row['Prediction_cleaned'] in row['genre']

df_classification_prueba_1['Prediction_in_genre'] = df_classification_prueba_1.apply(check_contains1, axis=1)

# Count how many times Column1 is in Column2
Coincidencias = df_classification_prueba_1['Prediction_in_genre'] .sum()

Coincidencias

47

#### **3.2.1. Modelo con fine tuning**

In [40]:
df_classification_for_finetuning=df_classification[['overview','classification(genre)']].iloc[0:10000]
df_classification_for_finetuning.head(5)

,overview,classification(genre)
0,"Overview: After dominating the boxing world, A...",Drama
1,Overview: Set more than a decade after the eve...,Science Fiction
2,Overview: While working underground to fix a w...,Animation
3,Overview: Through a series of unfortunate even...,Animation
4,Overview: Good-hearted teenager William always...,Action


In [41]:
def preprocess(ds):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    instructions = []

    # Iterar sobre las filas del DataFrame
    for index, row in ds.iterrows():
        # Concatenar 'overview' y 'classification_genre' en una instrucción
        instruction = f"{row['overview']}\n{row['classification(genre)']}"
        # Agregar la instrucción a la lista
        instructions.append(instruction)
    return instructions

list_ds = preprocess(df_classification_for_finetuning)
clean_ds = Dataset.from_dict({"data": list_ds})

In [42]:
def cross_validation(clean_ds):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    partition_ds = clean_ds.train_test_split(test_size=0.3, seed=13)
    return partition_ds
    ### FIN DEL CÓDIGO ###

partition_ds = cross_validation(clean_ds)

In [43]:
def tokenize(partition_ds):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    tokenizer = CodeGenTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
    def tokenize(sample):
      inputs = tokenizer(sample["data"], truncation=True, max_length=128)
      return {"input_ids": inputs["input_ids"]}
    token_ds = partition_ds.map(tokenize, remove_columns="data").with_format("torch")
    #
    return tokenizer, token_ds

tokenizer, token_ds = tokenize(partition_ds)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [44]:
def get_model():
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    config = CodeGenConfig.from_pretrained("Salesforce/codegen-350M-mono" ,n_ctx=128)
    model = CodeGenForCausalLM(config).to("cuda")
    return model
    ### FIN DEL CÓDIGO ###

model = get_model()

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

In [45]:
# FUNCIÓN CALIFICADA get_model:

def train_args():
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    args = TrainingArguments(
        output_dir="model",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",#"steps",
        num_train_epochs=5,
        warmup_steps=500,
        fp16=True,
        learning_rate=1e-4,
        save_strategy="epoch",
        save_total_limit=1,
    )
    return args
    ### FIN DEL CÓDIGO ###

args = train_args()


In [46]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=token_ds["train"],
    eval_dataset=token_ds["test"]
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,5.974651
2,6.886100,5.493163
3,5.295600,5.326640
4,4.544200,5.265824
5,3.906400,5.279836


TrainOutput(global_step=2190, training_loss=5.023638163527397, metrics={'train_runtime': 1529.9169, 'train_samples_per_second': 22.877, 'train_steps_per_second': 1.431, 'total_flos': 7649734727565312.0, 'train_loss': 5.023638163527397, 'epoch': 5.0})

In [50]:
def generate(text):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    pipe = pipeline(model="./model/checkpoint-2190/", task="text-generation")
    generated_text = pipe(text)
    return generated_text
    ### FIN DEL CÓDIGO ###

In [51]:
df_classification_prueba_2=df_classification.iloc[10000:].reset_index(drop=True)
df_classification_prueba_2["Prediction"] = None
print("Número de filas del dataframe: ",len(df_classification_prueba_2))

def limit_tokens(string, limit):
    tokens = string.split()
    limited_tokens = tokens[:limit]
    return ' '.join(limited_tokens)

Número de filas del dataframe:  52


In [62]:
import time
for i in range(len(df_classification_prueba_2)):
  if i%1==0:
    print("Step: ",i)
  overview_string=df_classification_prueba_2.iloc[i, 4]
  result = limit_tokens(overview_string, 25)
  prediction = generate(result+"\n")
  prediction_text = prediction[0]['generated_text']
  df_classification_prueba_2.loc[i, "Prediction"] = prediction_text
  only_answer=prediction_text.replace(result, '')
  df_classification_prueba_2.loc[i, "Prediction_cleaned"] = only_answer.split()[0].replace(",", "").replace(".", "")
  time.sleep(1)

Step:  0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Step:  1


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Step:  2


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Step:  50


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Step:  51


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [64]:
df_classification_prueba_2["Prediction_cleaned"] = df_classification_prueba_2["Prediction_cleaned"].str.replace('[.,!]', '', regex=True)
df_classification_prueba_2

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,classification(genre),Prediction,Prediction_cleaned
0,Tommy,03/26/1975,64.0,"Drama, Music, Fantasy","Overview: A psychosomatically deaf, dumb and b...","Oliver Reed, Frank, Ann-Margret, Nora, Roger D...",Tommy,Released,English,5000000.0,34251525.0,GB,Drama,"Overview: A psychosomatically deaf, dumb and b...",Family
1,Acts of Violence,01/12/2018,57.0,"Action, Crime, Thriller",Overview: When his fiancee is kidnapped by hum...,"Bruce Willis, Detective James Avery, Cole Haus...",Acts of Violence,Released,English,10700000.0,386790.0,US,Action,Overview: When his fiancee is kidnapped by hum...,Adventure
2,After Hours,09/13/1985,75.0,"Comedy, Thriller, Drama",Overview: An ordinary word processor has the w...,"Griffin Dunne, Paul Hackett, Rosanna Arquette,...",After Hours,Released,English,4500000.0,10609321.0,US,Comedy,Overview: An ordinary word processor has the w...,Thriller
3,Small Fry,11/23/2011,69.0,"Animation, Family, Comedy",Overview: A fast food restaurant mini variant ...,"Tom Hanks, Woody (voice), Tim Allen, Buzz Ligh...",Small Fry,Released,English,116400000.0,187299322.0,US,Animation,Overview: A fast food restaurant mini variant ...,Animation
4,Four of the Apocalypse,08/12/1975,65.0,"Western, Action","Overview: Four petty criminals, three men and ...","Fabio Testi, Stubby Preston, Lynne Frederick, ...",I quattro dell'apocalisse,Released,Italian,73340000.0,552832145.2,IT,Western,"Overview: Four petty criminals, three men and ...",Thriller
5,Love Wedding Repeat,04/10/2020,57.0,"Comedy, Romance",Overview: While trying to make his sister's we...,"Sam Claflin, Jack, Olivia Munn, Dina, Eleanor ...",Love Wedding Repeat,Released,English,128000000.0,419965614.6,GB,Comedy,Overview: While trying to make his sister's we...,Animation
6,Brüno,07/08/2009,56.0,Comedy,Overview: Flamboyantly gay Austrian television...,"Sacha Baron Cohen, Brüno, Gustaf Hammarsten, L...",Brüno,Released,English,42000000.0,138708527.0,AU,Comedy,Overview: Flamboyantly gay Austrian television...,Comedy
7,Vixen!,10/15/1968,53.0,Drama,Overview: Vixen lives in a Canadian mountain r...,"Garth Pillsbury, Tom Palmer, Harrison Page, Ni...",Vixen!,Released,English,73000.0,8000000.0,US,Drama,Overview: Vixen lives in a Canadian mountain r...,Drama
8,Lady Chatterley's Lover,09/06/2015,64.0,"Romance, Drama",Overview: An early-20th-century tale of love a...,"Richard Madden, Oliver Mellors, Holliday Grain...",Lady Chatterley's Lover,Released,English,102000000.0,254278668.2,GB,Romance,Overview: An early-20th-century tale of love a...,Drama
9,Felony,08/28/2014,57.0,"Mystery, Thriller, Crime",Overview: Three detectives become embroiled in...,"Joel Edgerton, Detective Malcolm Toohey, Jai C...",Felony,Released,English,169600000.0,453824304.6,AU,Mystery,Overview: Three detectives become embroiled in...,Drama


In [65]:
def check_contains2(row):
    return row['Prediction_cleaned'] in row['genre']

df_classification_prueba_2['Prediction_in_genre'] = df_classification_prueba_2.apply(check_contains2, axis=1)

# Count how many times Column1 is in Column2
Coincidencias = df_classification_prueba_2['Prediction_in_genre'] .sum()

Coincidencias

28